<a href="https://colab.research.google.com/github/Pavani3005/ViT_assignment/blob/main/q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -r '/content/sample_data'

In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm # For a nice progress bar

# Set a seed for reproducibility
torch.manual_seed(42)

# Check for GPU availability and set the device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# --- CONFIGURATION ---
IMG_SIZE = 32
PATCH_SIZE = 4
NUM_CLASSES = 10 # CIFAR-10 has 10 classes
EMB_DIM = 512    # Embedding dimension
NUM_HEADS = 8    # Number of attention heads
NUM_LAYERS = 6   # Number of Transformer encoder layers
HIDDEN_DIM = 2048 # Hidden dimension in the MLP
DROPOUT = 0.2

# --- TRAINING PARAMS ---
BATCH_SIZE = 256
LEARNING_RATE = 1e-4
NUM_EPOCHS = 100

In [ ]:
# --- DATA PREPARATION ---

# Define transformations for training data (with augmentation)
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.TrivialAugmentWide(),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])
])

# Define transformations for test data (only normalization)
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])
])

# Download datasets
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

100%|██████████| 170M/170M [00:13<00:00, 12.3MB/s]


In [ ]:
# --- MODEL ARCHITECTURE ---

class PatchEmbedding(nn.Module):
    """Converts an image into a sequence of patch embeddings."""
    def __init__(self):
        super().__init__()
        self.patcher = nn.Conv2d(
            in_channels=3,
            out_channels=EMB_DIM,
            kernel_size=PATCH_SIZE,
            stride=PATCH_SIZE
        )

    def forward(self, x):
        # x shape: [B, C, H, W] -> e.g., [256, 3, 32, 32]
        # Output shape: [B, EMB_DIM, H/P, W/P] -> e.g., [256, 512, 8, 8]
        x = self.patcher(x)

        # Flatten and transpose
        # Output shape: [B, EMB_DIM, N] -> e.g., [256, 512, 64]
        x = x.flatten(2)
        # Output shape: [B, N, EMB_DIM] -> e.g., [256, 64, 512]
        x = x.transpose(1, 2)
        return x

In [ ]:
class TransformerEncoder(nn.Module):
    """The standard Transformer encoder block."""
    def __init__(self):
        super().__init__()
        # Attention block
        self.norm1 = nn.LayerNorm(EMB_DIM)
        self.attn = nn.MultiheadAttention(EMB_DIM, NUM_HEADS, dropout=DROPOUT, batch_first=True)
        self.dropout1 = nn.Dropout(DROPOUT)

        # MLP block
        self.norm2 = nn.LayerNorm(EMB_DIM)
        self.mlp = nn.Sequential(
            nn.Linear(EMB_DIM, HIDDEN_DIM),
            nn.GELU(),
            nn.Dropout(DROPOUT),
            nn.Linear(HIDDEN_DIM, EMB_DIM),
            nn.Dropout(DROPOUT)
        )

    def forward(self, x):
        # Attention block with residual connection
        x_norm = self.norm1(x)
        attn_out, _ = self.attn(x_norm, x_norm, x_norm)
        x = x + self.dropout1(attn_out)

        # MLP block with residual connection
        x_norm = self.norm2(x)
        mlp_out = self.mlp(x_norm)
        x = x + mlp_out

        return x

In [ ]:
class VisionTransformer(nn.Module):
    """The full Vision Transformer model."""
    def __init__(self):
        super().__init__()
        num_patches = (IMG_SIZE // PATCH_SIZE) ** 2

        self.patch_embedding = PatchEmbedding()

        # Learnable CLS token
        self.cls_token = nn.Parameter(torch.randn(1, 1, EMB_DIM))

        # Learnable positional embeddings
        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, EMB_DIM))

        # Stack of Transformer encoders
        self.encoders = nn.Sequential(*[TransformerEncoder() for _ in range(NUM_LAYERS)])

        # Final classification head
        self.norm = nn.LayerNorm(EMB_DIM)
        self.classifier = nn.Linear(EMB_DIM, NUM_CLASSES)

    def forward(self, x):
        # 1. Create patch embeddings
        x = self.patch_embedding(x)

        # 2. Prepend CLS token
        # Expand cls_token to match the batch size
        cls_tokens = self.cls_token.expand(x.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)

        # 3. Add positional embeddings
        x = x + self.pos_embedding

        # 4. Pass through Transformer encoders
        x = self.encoders(x)

        # 5. Get the CLS token output for classification
        # We only use the output corresponding to the first token (CLS token)
        cls_output = x[:, 0]

        # 6. Pass through the final norm and classifier
        cls_output = self.norm(cls_output)
        out = self.classifier(cls_output)

        return out

# Instantiate the model and move it to the GPU
model = VisionTransformer().to(device)
print(model)

VisionTransformer(
  (patch_embedding): PatchEmbedding(
    (patcher): Conv2d(3, 512, kernel_size=(4, 4), stride=(4, 4))
  )
  (encoders): Sequential(
    (0): TransformerEncoder(
      (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
      )
      (dropout1): Dropout(p=0.2, inplace=False)
      (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (mlp): Sequential(
        (0): Linear(in_features=512, out_features=2048, bias=True)
        (1): GELU(approximate='none')
        (2): Dropout(p=0.2, inplace=False)
        (3): Linear(in_features=2048, out_features=512, bias=True)
        (4): Dropout(p=0.2, inplace=False)
      )
    )
    (1): TransformerEncoder(
      (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=5

In [ ]:
# --- TRAINING SETUP ---

# Loss function
criterion = nn.CrossEntropyLoss()
# Optimizer
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.1)
# Learning rate scheduler
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

# --- TRAINING & EVALUATION FUNCTIONS ---

def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_samples = 0

    progress_bar = tqdm(loader, desc="Training")
    for inputs, labels in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        correct_preds += (predicted == labels).sum().item()

        progress_bar.set_postfix(loss=loss.item(), acc=f"{(predicted == labels).sum().item()/labels.size(0):.2f}")

    epoch_loss = running_loss / total_samples
    epoch_acc = correct_preds / total_samples
    return epoch_loss, epoch_acc


def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct_preds = 0
    total_samples = 0

    with torch.no_grad():
        progress_bar = tqdm(loader, desc="Evaluating")
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_preds += (predicted == labels).sum().item()

            progress_bar.set_postfix(loss=loss.item())

    epoch_loss = running_loss / total_samples
    epoch_acc = correct_preds / total_samples
    return epoch_loss, epoch_acc

In [ ]:
# --- MAIN TRAINING LOOP ---

best_test_acc = 0.0

print("Starting training...")
for epoch in range(NUM_EPOCHS):
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)

    # Step the scheduler
    scheduler.step()

    print(f"Epoch {epoch+1}/{NUM_EPOCHS} | "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
          f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")

    # Save the best model
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        torch.save(model.state_dict(), 'best_vit_cifar10.pth')
        print(f"New best model saved with accuracy: {best_test_acc:.4f}")

print("Training finished!")
print(f"Best Test Accuracy: {best_test_acc:.4f}")

Starting training...


Evaluating: 100%|██████████| 40/40 [00:08<00:00,  4.65it/s, loss=1.56]


Epoch 1/100 | Train Loss: 2.0402, Train Acc: 0.2517 | Test Loss: 1.6046, Test Acc: 0.4280
New best model saved with accuracy: 0.4280


Evaluating: 100%|██████████| 40/40 [00:08<00:00,  4.85it/s, loss=1.23]


Epoch 2/100 | Train Loss: 1.8060, Train Acc: 0.3501 | Test Loss: 1.4088, Test Acc: 0.4993
New best model saved with accuracy: 0.4993


...

Epoch 98/100 | Train Loss: 0.7588, Train Acc: 0.7297 | Test Loss: 0.5871, Test Acc: 0.8045


Evaluating: 100%|██████████| 40/40 [00:08<00:00,  4.85it/s, loss=0.36]


Epoch 99/100 | Train Loss: 0.7562, Train Acc: 0.7338 | Test Loss: 0.5865, Test Acc: 0.8041


Evaluating: 100%|██████████| 40/40 [00:08<00:00,  4.86it/s, loss=0.36]

Epoch 100/100 | Train Loss: 0.7532, Train Acc: 0.7326 | Test Loss: 0.5866, Test Acc: 0.8041
Training finished!
Best Test Accuracy: 0.8064


# Vision Transformer (ViT) for CIFAR-10 Classification

This notebook implements a Vision Transformer model from scratch in PyTorch to classify images from the CIFAR-10 dataset.

## Overview

The notebook covers the following steps:

1.  **Setup**: Imports necessary libraries and sets up the device (GPU or CPU).
2.  **Configuration**: Defines hyperparameters for the model and training process.
3.  **Data Preparation**: Downloads, transforms, and loads the CIFAR-10 dataset using PyTorch DataLoaders. Data augmentation is applied to the training set.
4.  **Model Architecture**: Defines the Vision Transformer architecture, including:
    *   `PatchEmbedding`: Converts input images into a sequence of flattened patches and projects them into a higher-dimensional space.
    *   `TransformerEncoder`: Implements a standard Transformer encoder layer with multi-head attention and an MLP block.
    *   `VisionTransformer`: Combines the patch embedding, learnable CLS token, positional embeddings, and a stack of Transformer encoders with a final classification head.
5.  **Training Setup**: Defines the loss function (Cross-Entropy), optimizer (AdamW), and learning rate scheduler (Cosine Annealing).
6.  **Training & Evaluation Functions**: Implements functions for training one epoch and evaluating the model on the test set.
7.  **Main Training Loop**: Runs the training process for a specified number of epochs, saves the best model based on test accuracy, and prints training and evaluation metrics per epoch.

## Requirements

*   PyTorch
*   Torchvision
*   Tqdm

These libraries are typically pre-installed in Google Colab environments.

## Usage

1.  **Run all cells**: Execute the cells sequentially from top to bottom.
2.  **Monitor Training**: Observe the training progress and metrics printed in the output of the training loop cell. The best test accuracy achieved will be reported at the end.
3.  **Best Model**: The best performing model weights will be saved to a file named `best_vit_cifar10.pth`.

## Configuration

You can adjust the model and training hyperparameters in the "Configuration" cell (e.g., `IMG_SIZE`, `PATCH_SIZE`, `NUM_EPOCHS`, `LEARNING_RATE`, etc.) to experiment with different settings.

## Dataset

The notebook uses the CIFAR-10 dataset, which consists of 60,000 32x32 color images in 10 classes, with 6,000 images per class. There are 50,000 training images and 10,000 test images.